In [52]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(action='ignore') 

In [53]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [54]:
from efficientnet_pytorch import EfficientNet
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # 디바이스 설정
print(device)

cuda:0


In [55]:
torch.cuda.is_available()

True

In [56]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS': 15,
    'LEARNING_RATE':0.001,
    'BATCH_SIZE':16,
    'SEED':42
}

In [57]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])

In [58]:
df = pd.read_csv('./train.csv')

In [59]:
def get_labels(df):
    return df.iloc[:,2:].values

In [60]:
train, test, train_labels, test_labels = train_test_split(df, get_labels(df),
                                                     test_size=0.1, shuffle=True,random_state=CFG['SEED'])
# val, test, val_labels, test_labels = train_test_split(test, get_labels(test),
#                                                      test_size=0.5, shuffle=True, random_state=CFG['SEED'])

In [61]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = torch.FloatTensor(self.label_list[index])
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [62]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [63]:
train_dataset = CustomDataset(train['img_path'].values, train_labels, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

# val_dataset = CustomDataset(val['img_path'].values, val_labels, test_transform)
# val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

test_dataset = CustomDataset(test['img_path'].values, test_labels, test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [64]:
class efficient_Model(nn.Module):
    def __init__(self, num_classes=10):
        super(efficient_Model, self).__init__()
        #self.backbone = models.convnext_large(pretrained=True)
        
        #self.backbone = models.efficientnet_b5(pretrained=True)
       
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b7')
        self.drop = nn.Dropout(p=0.2)
        self.FC = nn.Linear(1000, 10)
        
        
    def forward(self, x):
        x = F.silu(x)

    
        x = F.silu(self.efficientnet(x))

        x = self.drop(x)
        
        x = torch.sigmoid(self.FC(x))
        return x


In [65]:
class convnext_Model(nn.Module):
    def __init__(self, num_classes=10):
        super(convnext_Model, self).__init__()
        #self.backbone = models.convnext_large(pretrained=True)
        
        #self.backbone = models.efficientnet_b5(pretrained=True)
        
        
        self.backbone = models.convnext_large(pretrained=True)
        self.drop = nn.Dropout(p=0.2)
        self.FC = nn.Linear(1000, 10)
        
        
    def forward(self, x):
        x = F.silu(x)

    
        x = F.silu(self.efficientnet(x))

        x = self.drop(x)
        
        x = torch.sigmoid(self.FC(x))
        return x

In [66]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    val_acc = []
    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            probs = model(imgs)
            
            loss = criterion(probs, labels)
            
            probs  = probs.cpu().detach().numpy()
            labels = labels.cpu().detach().numpy()
            preds = probs > 0.5
            batch_acc = (labels == preds).mean()
            
            val_acc.append(batch_acc)
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_acc = np.mean(val_acc)
    
    return _val_loss, _val_acc

In [67]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.BCELoss().to(device)
    #criterion = nn.CrossEntropyLoss().to(device)
    #criterion = nn.NLLLoss().to(device)
    best_val_acc = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_acc = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val ACC : [{_val_acc:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_val_acc)
            
        if best_val_acc < _val_acc:
            best_val_acc = _val_acc
            best_model = model
    
    return best_model

In [68]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

best_models = [] # 폴드별로 가장 validation acc가 높은 모델 저장

for fold, (train_idx,val_idx) in enumerate(kfold.split(np.arange(len(train_dataset)))):

    print('Fold {}'.format(fold + 1))

    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)
    
    # Define data loaders for training and testing data in this fold
    #test데이터 10% 빼놓고 남은 90%을 이용해서 훈련-loss계산하는 식으로 진행함
    trainloader = torch.utils.data.DataLoader(
                      train_dataset, 
                      batch_size=CFG['BATCH_SIZE'], sampler=train_subsampler)
    
    valloader = torch.utils.data.DataLoader(
                      train_dataset,
                      batch_size=CFG['BATCH_SIZE'], sampler=val_subsampler)
    
    #각 폴드마다 모델은 새로 갱신
    model = efficient_Model()
    
    optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

    #각 fold한 데이터셋마다 모델를 적용해서 가장 val_acc가 적은 모델을 선택함
    train_model= train(model,optimizer,trainloader,valloader,scheduler,device) 
    
    #각 데이터셋마다 최고의 모델을 한개씩 집어 넣음
    #kfold를 5개로 했으니까 총 5개의 모델이 들어있음
    best_models.append(train_model)

Fold 1
Loaded pretrained weights for efficientnet-b7


  0%|          | 0/1485 [00:00<?, ?it/s]

  0%|          | 0/372 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.25317] Val Loss : [0.13114] Val ACC : [0.95077]


  0%|          | 0/1485 [00:00<?, ?it/s]

...

  0%|          | 0/1485 [00:00<?, ?it/s]

  0%|          | 0/372 [00:00<?, ?it/s]

Epoch [15], Train Loss : [0.01099] Val Loss : [0.03533] Val ACC : [0.99002]
Fold 5
Loaded pretrained weights for efficientnet-b7


  0%|          | 0/1485 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 18.00 MiB (GPU 0; 12.00 GiB total capacity; 11.08 GiB already allocated; 0 bytes free; 11.31 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

best_models1 = [] # 폴드별로 가장 validation acc가 높은 모델 저장

for fold, (train_idx,val_idx) in enumerate(kfold.split(np.arange(len(train_dataset)))):

    print('Fold {}'.format(fold + 1))

    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)
    
    # Define data loaders for training and testing data in this fold
    #test데이터 10% 빼놓고 남은 90%을 이용해서 훈련-loss계산하는 식으로 진행함
    trainloader = torch.utils.data.DataLoader(
                      train_dataset, 
                      batch_size=CFG['BATCH_SIZE'], sampler=train_subsampler)
    
    valloader = torch.utils.data.DataLoader(
                      train_dataset,
                      batch_size=CFG['BATCH_SIZE'], sampler=val_subsampler)
    
    #각 폴드마다 모델은 새로 갱신
    model = convnext_Model()
    
    optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

    #각 fold한 데이터셋마다 모델를 적용해서 가장 val_acc가 적은 모델을 선택함
    train_model= train(model,optimizer,trainloader,valloader,scheduler,device) 
    
    #각 데이터셋마다 최고의 모델을 한개씩 집어 넣음
    #kfold를 5개로 했으니까 총 5개의 모델이 들어있음
    best_models1.append(train_model)

In [ ]:
# model = Model()

# model.to(device)

# optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

# optimizer = torch.optim.AdamW(model.parameters(),lr = 1e-3)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=50, T_mult=2, eta_min=0.001)
# criterion = torch.nn.CrossEntropyLoss()

In [ ]:
#infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for imgs,labels in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            probs = model(imgs)

            #확률값을 이용해서 softvoting할 예정이므로 확률값을 저장함
            probs  = probs.cpu().detach().numpy()
            preds = probs > 0.5
            preds = preds.astype(int)
            predictions += probs.tolist()
    return predictions

In [ ]:
preds_list = []
for m in best_models:
    pred = inference(m,test_loader,device)
    preds_list.append(pred)

  0%|          | 0/207 [00:00<?, ?it/s]

[[1.0,
  0.0013927161926403642,
  0.9999257326126099,
  0.999612033367157,
  0.9752447605133057,
  0.9999990463256836,
  0.9984379410743713,
  0.11506610363721848,
  0.9683306813240051,
  0.014966250397264957],
 [1.0,
  0.9999964237213135,
  1.0,
  1.0,
  1.0,
  0.9999998807907104,
  9.730272837460419e-11,
  1.0440682984835803e-08,
  1.0,
  4.618037507952977e-08],
 [0.0001058107809512876,
  0.9999233484268188,
  1.6625902432565454e-08,
  1.0001003358794947e-09,
  0.9999954700469971,
  0.0005698155728168786,
  0.9999955892562866,
  3.501959833229762e-11,
  1.3849419246980688e-06,
  0.9999949932098389],
 [8.41607317170201e-09,
  0.0001664261653786525,
  0.9999904632568359,
  1.0,
  0.999980092048645,
  1.0,
  1.0,
  7.811321034445484e-12,
  1.0,
  0.9999995231628418],
 [1.0,
  3.34436606408417e-07,
  1.0,
  8.17915873918551e-11,
  0.9999995231628418,
  1.0,
  1.0,
  1.5303323053217355e-09,
  4.106304913875647e-05,
  8.418786556774194e-09],
 [9.585851535286183e-15,
  1.0,
  0.999999761581

In [ ]:
preds_list1 = []
for m in best_models1:
    pred = inference(m,test_loader,device)
    preds_list1.append(pred)

In [ ]:
predictions_array1 = np.concatenate(preds_list, axis=2)
predictions_array2 = np.concatenate(preds_list1, axis=2)
predictions_array_final = np.concatenate([predictions_array1, predictions_array2], axis=2)
predictions_mean = predictions_array_final.mean(axis = 2)
# 평균 값이 0.5보다 클 경우 1 작으면 0
predictions_mean = (predictions_mean > 0.5) * 1

In [ ]:
a=[]
for idx in range(len(predictions_mean)):
  if (test_labels[idx] == predictions_mean[idx]).all():
    a.append(1)
  else:
    a.append(0)
a = np.array(a)

accuracy = a.sum() / len(a)
print('Accuracy : ',accuracy)

Accuracy :  0.0


In [ ]:
#torch.save(model, './densenet_model')

# test 데이터 성능평가

In [ ]:
test = pd.read_csv('./test.csv')

In [ ]:
test

,id,img_path
0,TEST_00000,./test/TEST_00000.jpg
1,TEST_00001,./test/TEST_00001.jpg
2,TEST_00002,./test/TEST_00002.jpg
3,TEST_00003,./test/TEST_00003.jpg
4,TEST_00004,./test/TEST_00004.jpg
...,...,...
1455,TEST_01455,./test/TEST_01455.jpg
1456,TEST_01456,./test/TEST_01456.jpg
1457,TEST_01457,./test/TEST_01457.jpg
1458,TEST_01458,./test/TEST_01458.jpg


In [ ]:
from rembg.bg import remove as remove_bg
class TetsDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        image = remove_bg(image) ##바뀐부분
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = torch.FloatTensor(self.label_list[index])
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [ ]:
test_dataset = TetsDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            probs = model(imgs)

            probs  = probs.cpu().detach().numpy()
            preds = probs > 0.5
            preds = preds.astype(int)
            predictions += preds.tolist()
    return predictions

In [ ]:
preds_list = []
for m in best_models:
    pred = inference(m,test_loader,device)
    preds_list.append(pred)

In [ ]:
preds_list1 = []
for m in best_models1:
    pred = inference(m,test_loader,device)
    preds_list1.append(pred)

In [ ]:
predictions_array1 = np.concatenate(preds_list, axis=2)
predictions_array2 = np.concatenate(preds_list1, axis=2)
predictions_array_final = np.concatenate([predictions_array1, predictions_array2], axis=2)
predictions_mean = predictions_array_final.mean(axis = 2)
# 평균 값이 0.5보다 클 경우 1 작으면 0
predictions_mean = (predictions_mean > 0.5) * 1

In [ ]:
#preds = inference(model, test_loader, device)

  0%|          | 0/92 [00:00<?, ?it/s]

In [ ]:
submit = pd.read_csv('./sample_submission.csv')

In [ ]:
submit.iloc[:,1:] = predictions_mean
submit.head()

,id,A,B,C,D,E,F,G,H,I,J
0,TEST_00000,1,1,1,0,0,0,1,0,1,0
1,TEST_00001,1,1,1,0,0,1,1,0,0,0
2,TEST_00002,1,1,1,0,0,1,0,0,1,0
3,TEST_00003,1,1,1,0,0,0,0,0,1,1
4,TEST_00004,1,1,1,0,0,0,1,0,1,0


In [ ]:
submit.to_csv('./baseline.csv', index=False)